In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
from mpl_toolkits.mplot3d import Axes3D
import netCDF4
import numpy
import os
import pandas
from PIL import Image
import random
import readline
import scipy
from scipy import fft
from scipy import linalg
from scipy import stats
from scipy.stats import poisson, ttest_ind
import seaborn
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import normalize
from statsmodels.tsa.ar_model import AutoReg
import xarray as xr

In [2]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Data/RCP_Files_V4/'
Output_Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Data/Output_Files_V6/'

In [3]:
# Open File
def Create_DF(File):
    Data = open(File, 'r')
    Rows = []
#
# Organize Data
    for Line in Data:
        Rows.append(Line.strip())
    Storm_Code = []
    Storm_List_Orig = []
    for i in range(len(Rows)):
        if Rows[i][0:5] == 'start':
            Code = Rows[i][41:45]
            Storm_List_Orig.append(Code)
        else:
            Storm_Code.append(Code)
    Array = numpy.zeros((13, len(Rows)-len(Storm_List_Orig)))
    Time = []
    k = -1
    for i in range(len(Rows)):
#        print (i)
        if Rows[i][0:5] == 'start':
            k += 1
        else:
            l = len(Rows[i]) - 100
            Array[0][i-k-1] = float(Rows[i][0:6+l])
            Array[1][i-k-1] = float(Rows[i][8+l:14+l])
            Array[2][i-k-1] = float(Rows[i][17+l:24+l])
            Array[3][i-k-1] = float(Rows[i][26+l:31+l])
            Array[4][i-k-1] = float(Rows[i][34+l:41+l])
            Array[5][i-k-1] = float(Rows[i][44+l:51+l])
            Array[6][i-k-1] = float(Rows[i][54+l:61+l])
            Array[7][i-k-1] = float(Rows[i][64+l:71+l])
            Array[8][i-k-1] = float(Rows[i][74+l:81+l])
            Time.append(datetime.datetime(year=int(Rows[i][84+l:88+l]), month=int(Rows[i][90+l:92+l]), \
            day=int(Rows[i][94+l:96+l]), hour=int(Rows[i][98+l:100+l])))
#
# Create DataFrame to Store Data
    DF_All = pandas.DataFrame({"Orig Code": Storm_Code, "Lon": Array[0], "Lat": Array[1], "SLP(hPa)": Array[2], \
    "Winds(m/s)": Array[3], "Dist(m)": Array[4], "Angle": Array[5], "B": Array[6], "VLT": Array[7], "VUT": Array[8], \
    "Orig Time(Z)": Time})
#
# Remove Data Points Missing Phase Space Parameter Values (-999.0)
    DF = DF_All[(DF_All["B"] > -728) & (DF_All["VLT"] > -728) & (DF_All["VUT"] > -728)].reset_index()
    return (DF, Storm_List_Orig)

In [4]:
# Combine The Three Datasets Into One DataFrame of 90 or 93 Years For Each Scenario
def Combine_DF(DF_A, DF_B, DF_C, Orig_List_A, Orig_List_B, Orig_List_C, Model):
# Change Date For Simplicity of Analysis
# Control Assigned to Years in the 20th Century
# Each Year Within the Century are Independent
    if Model == "Control":
        Year_Start = 1900
# RCP4.5 Assigned to Years in the 21st Century
    elif Model == "RCP45":
        Year_Start = 2000
# RCP8.5 Assigned to Years in the 22nd Century
    elif Model == "RCP85":
        Year_Start = 2100
# Create Empty Lists
    Origi_List_A = []
    Origi_List_B = []
    Origi_List_C = []
    New_List_A = []
    New_List_B = []
    New_List_C = []
    New_Time = []
    New_Code = []
    ABC = []
    Orig_Combine = []
    New_Combine = []
# Dataset A
    Year = Year_Start
    Orig_Year = DF_A["Orig Time(Z)"][0].year
    Year_Diff_A = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_A)):
        try:
            Code, Year, Count = Update_Code(DF_A, Orig_List_A[k], Year_Diff_A, Year, Count)
            Origi_List_A.append(Orig_List_A[k])
            New_List_A.append(Code)
        except:
            Kinen = 728
# Dataset B
    Year += 1
    Orig_Year = DF_B["Orig Time(Z)"][0].year
    Year_Diff_B = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_B)):
        try:
            Code, Year, Count = Update_Code(DF_B, Orig_List_B[k], Year_Diff_B, Year, Count)
            Origi_List_B.append(Orig_List_B[k])
            New_List_B.append(Code)
        except:
            Kinen = 728
# Dataset C
    Year += 1
    Orig_Year = DF_C["Orig Time(Z)"][0].year
    Year_Diff_C = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_C)):
        try:
            Code, Year, Count = Update_Code(DF_C, Orig_List_C[k], Year_Diff_C, Year, Count)
            Origi_List_C.append(Orig_List_C[k])
            New_List_C.append(Code)
        except:
            Kinen = 728
#
# Combine Into New Time and Codes List
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_A, Origi_List_A, New_List_A, Year_Diff_A, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "A")
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_B, Origi_List_B, New_List_B, Year_Diff_B, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "B")
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_C, Origi_List_C, New_List_C, Year_Diff_C, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "C")
#
# Concatenate Other Variables
    Vars = ["Lon", "Lat", "SLP(hPa)", "Winds(m/s)", "Dist(m)", "Angle", "B", "VLT", "VUT"]
    Total_Length = len(New_Code)
    Array = numpy.zeros((9,Total_Length))
    for n in range(len(Vars)):
        List = []
        for k in range(len(DF_A)):
            List.append(DF_A[Vars[n]][k])
        for k in range(len(DF_B)):
            List.append(DF_B[Vars[n]][k])
        for k in range(len(DF_C)):
            List.append(DF_C[Vars[n]][k])
        Array[n] = List
#
# Create DataFrame to Store Data
    DF_Combine = pandas.DataFrame({"Code": New_Code, "Lon": Array[0], "Lat": Array[1], "SLP(hPa)": Array[2], \
    "Winds(m/s)": Array[3], "B": Array[6], "VLT": Array[7], "VUT": Array[8], "Time(Z)": New_Time})
    Codes_DF = pandas.DataFrame({"ABC": ABC, "Orig Code": Orig_Combine, "New Code": New_Combine})
    Year_Diffs = numpy.array([Year_Diff_A, Year_Diff_B, Year_Diff_C])
    return (DF_Combine, Codes_DF, Year_Diffs)

In [5]:
# Update Storm Code to Contain Year and Storm Number
def Update_Code(DF, Orig_Code, Year_Diff_A, Year, Count):
    DF_Storm = DF[DF["Orig Code"] == Orig_Code].reset_index()
    Orig_Year = DF_Storm["Orig Time(Z)"][0].year
    Time_Year = Orig_Year + Year_Diff_A
    if Time_Year == Year:
        Count += 1
    else:
        Year += 1
        Count = 1
    if Count < 10:
        Code = "TC"+str(Year)+"0"+str(Count)
    else:
        Code = "TC"+str(Year)+str(Count)
    return (Code, Year, Count)

In [6]:
# Combine Lists
def Combine_Lists(DF, Orig_List, New_List, Year_Diff, New_Code, New_Time, ABC, Orig_Combine, New_Combine, Y):
    for i in range(len(DF)):
        for l in range(len(Orig_List)):
# Update Storm Code
            if DF["Orig Code"][i] == Orig_List[l]:
                New_Code.append(New_List[l])
# Update Year
                Orig_Time = DF["Orig Time(Z)"][i]
                New_Time.append(Update_Year(Orig_Time, Year_Diff))
    for m in range(len(Orig_List)):
        ABC.append(Y)
        Orig_Combine.append(Orig_List[m])
        New_Combine.append(New_List[m])
    return (New_Code, New_Time, ABC, Orig_Combine, New_Combine)

In [7]:
# Change Year of Data
def Update_Year(Orig_Time, Year_Diff):
    Year_Update = Orig_Time.year + Year_Diff
# Febuary 29 Problems
    if Orig_Time.month == 2 and Orig_Time.day == 29:
        New_Time = Orig_Time.replace(year=Year_Update, day=28)
    else:
        New_Time = Orig_Time.replace(year=Year_Update)
    return (New_Time)

In [8]:
# Open Name List File
def Open_Name_List(File):
    Name_File = pandas.read_csv(File)
    Headings = list(Name_File.columns)
    Name_List = []
    for i in range(len(Headings)):
        for j in range(len(Name_File)):
            Name_List.append(Name_File[Headings[i]][j])
    return (Name_List)

In [9]:
# Assign Names to Each Cyclone Code
def Assign_Name(Name_List, Codes, DF):
    Storm_Names = []
    for i in range(len(Codes)):
        Name = Name_List[int(i%len(Name_List))]
        Storm_Names.append(Name)
    Codes["Name"] = Storm_Names
    DF_Names = []
    for k in range(len(DF)):
        Code = DF["Code"][k]
        Name = list(Codes[Codes["New Code"] == Code]["Name"])[0]
        DF_Names.append(Name)
    DF.insert(1, "Name", DF_Names)
    return (Codes, DF)

In [10]:
# Function to Find Distance Between Two Points
def Find_Distance(y1, y2, x1, x2):
    Start_Lat = y1 * numpy.pi / 180
    End_Lat = y2 * numpy.pi / 180
    Start_Lon = x1 * numpy.pi / 180
    End_Lon = x2 * numpy.pi / 180
    Lat_Diff = End_Lat - Start_Lat
    Lon_Diff = End_Lon - Start_Lon
    Earth_Rad = 6378
    Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
    numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
    return (Distance)

In [11]:
# Find a Specific Storm Within the DataFrame
def Find_Storm(DF, Code):
    DF_Storm = DF[DF["Code"] == Code].reset_index()
    return (DF_Storm)

In [12]:
# Open Name List
Name_List = Open_Name_List('Storm_Name_List.csv')

In [13]:
# Apply Functions to Open Datasets
Control_A_Data_V0, Control_A_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.control.002_avg_V4')
Control_B_Data_V0, Control_B_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.control.003_avg_V4')
Control_C_Data_V0, Control_C_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.control.004_avg_V4')

In [14]:
# Process Control DataFrames
Control_Data_V1, Control_Codes_V1, Control_Year_Diffs = \
Combine_DF(Control_A_Data_V0, Control_B_Data_V0, Control_C_Data_V0, \
Control_A_Storm_List_V0, Control_B_Storm_List_V0, Control_C_Storm_List_V0, "Control")
Control_Codes_V2, Control_Data_V2 = Assign_Name(Name_List, Control_Codes_V1, Control_Data_V1)

In [15]:
# Apply Functions to Open Datasets
RCP45_A_Data_V0, RCP45_A_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.rcp45.001_avg_V4')
RCP45_B_Data_V0, RCP45_B_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.rcp45.002_avg_V4')
RCP45_C_Data_V0, RCP45_C_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.rcp45.003_avg_V4')

In [16]:
# Process RCP4.5 DataFrames
RCP45_Data_V1, RCP45_Codes_V1, RCP45_Year_Diffs = \
Combine_DF(RCP45_A_Data_V0, RCP45_B_Data_V0, RCP45_C_Data_V0, \
RCP45_A_Storm_List_V0, RCP45_B_Storm_List_V0, RCP45_C_Storm_List_V0, "RCP45")
RCP45_Codes_V2, RCP45_Data_V2 = Assign_Name(Name_List, RCP45_Codes_V1, RCP45_Data_V1)

In [17]:
# Apply Functions to Open Datasets
RCP85_A_Data_V0, RCP85_A_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.rcp85.001_avg_V4')
RCP85_B_Data_V0, RCP85_B_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.rcp85.003_avg_V4')
RCP85_C_Data_V0, RCP85_C_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.rcp85.004_avg_V4')

In [18]:
# Process RCP8.5 DataFrames
RCP85_Data_V1, RCP85_Codes_V1, RCP85_Year_Diffs = \
Combine_DF(RCP85_A_Data_V0, RCP85_B_Data_V0, RCP85_C_Data_V0, \
RCP85_A_Storm_List_V0, RCP85_B_Storm_List_V0, RCP85_C_Storm_List_V0, "RCP85")
RCP85_Codes_V2, RCP85_Data_V2 = Assign_Name(Name_List, RCP85_Codes_V1, RCP85_Data_V1)

In [19]:
# Check For Repeated Entries
def Repeated_Check(DF):
    for i in range(len(DF)):
        Lat = DF["Lat"][i]
        Lon = DF["Lon"][i]
        Time = DF["Time(Z)"][i]
        DF_Repeat = DF[(DF["Lat"] == Lat) & (DF["Lon"] == Lon) & (DF["Time(Z)"] == Time)]
# Deleted Repeated Entries
        if len(DF_Repeat) > 1:
            print (DF["Code"][i], Time)

In [20]:
# Create DF With Extratropical Transition Information of Each Storm
def Storm_Info_DF(DF, Codes):
    Trans_Types = []
    Birth_Times = []
    Start_Times = []
    End_Times = []
    Death_Times = []
    Track_Discons = []
    for i in range(len(Codes)):
        DF_Storm_Orig = DF[DF["Code"] == Codes["New Code"][i]].reset_index()
        DF_Storm, Track_Discons = DF_Fixing(DF_Storm_Orig, Track_Discons)
        Trans_Type, Start_Time_k, End_Time_k = ET_Function(DF_Storm)
        Birth_Time = DF_Storm["Time(Z)"][0]
        if Start_Time_k > 0:
            Start_Time = DF_Storm["Time(Z)"][Start_Time_k]
        else:
            Start_Time = numpy.nan
        if End_Time_k > 0:
            End_Time = DF_Storm["Time(Z)"][End_Time_k]
        else:
            End_Time = numpy.nan
        Death_Time = DF_Storm["Time(Z)"][len(DF_Storm)-1]
        Birth_Times.append(Birth_Time)
        Start_Times.append(Start_Time)
        End_Times.append(End_Time)
        Death_Times.append(Death_Time)
        Trans_Types.append(Trans_Type)
        if i == 0:
            DF_Fixed = DF_Storm.copy()
        else:
            DF_Fixed = pandas.concat([DF_Fixed, DF_Storm])
    DF_Fixed = DF_Fixed.reset_index()
    DF_Fixed = DF_Fixed.drop("index", axis=1)
#    print (Track_Discons)
#
# Create Storms DF
    Storms_DF_Orig = pandas.DataFrame({"Code": Codes["New Code"], "Name": Codes["Name"], \
    "Trans Type": Trans_Types, "Genesis Time": Birth_Times, \
    "ET Begin Time": Start_Times, "ET Complete Time": End_Times, "Dissipate Time": Death_Times})
    return (DF_Fixed, Storms_DF_Orig)

In [21]:
# Function For Check For Time Discontinuity
def Time_Discontinuity(DF_Storm):
    Time_Continuous = 0
    for k in range(len(DF_Storm)):
        if k > 0 and Time_Continuous == 0:
#            print (k)
# If Over 24 Hours of Missing Data
            if DF_Storm["Time(Z)"][k] - DF_Storm["Time(Z)"][k-1] >= datetime.timedelta(hours=24):
# Divide Array Into Before The Time Discontinuity and After It
                Before = DF_Storm["Time(Z)"][k-1]
                After = DF_Storm["Time(Z)"][k]
                DF_Before = DF_Storm[DF_Storm["Time(Z)"] <= Before].reset_index()
                DF_After = DF_Storm[DF_Storm["Time(Z)"] >= After].reset_index()
# Choose the Part with the Lowest Min SLP to Keep
                if numpy.min(DF_After["SLP(hPa)"]) <= numpy.min(DF_Before["SLP(hPa)"]) and \
                len(DF_After) > len(DF_Before):
                    DF_Storm_Updated = DF_After
                else:
                    DF_Storm_Updated = DF_Before
                Time_Continuous = 1
                DF_Storm_Updated = DF_Storm_Updated.drop("level_0", axis=1)
# Repeat Until No More Time Discontinuity
    if Time_Continuous == 0:
        DF_Storm_Updated = DF_Storm.copy()
    return (DF_Storm_Updated)

In [22]:
# Function For Check For Track Discontinuity
def Track_Discontinuity(DF_Storm):
    Track_Continuous = 0
    Track_Discon = []
    for k in range(len(DF_Storm)):
        if k > 1 and Track_Continuous == 0:
# Calculate Distance Between Each 6 Hour Interval
            Distance_1 = Find_Distance(DF_Storm["Lat"][k], DF_Storm["Lat"][k-1], \
            DF_Storm["Lon"][k], DF_Storm["Lon"][k-1])
            Distance_2 = Find_Distance(DF_Storm["Lat"][k-1], DF_Storm["Lat"][k-2], \
            DF_Storm["Lon"][k-1], DF_Storm["Lon"][k-2])
# If Distance >= 800km and Distance Over 5 Times Further Than Previous Time Interval
            if Distance_1 >= 800 and Distance_1 >= Distance_2 * 5:
                Track_Discon = [DF_Storm["Code"][0], DF_Storm["Time(Z)"][k]]
# Divide Array Into Before The Track Discontinuity and After It
                Before = DF_Storm["Time(Z)"][k-1]
                After = DF_Storm["Time(Z)"][k]
                DF_Before = DF_Storm[DF_Storm["Time(Z)"] <= Before].reset_index()
                DF_After = DF_Storm[DF_Storm["Time(Z)"] >= After].reset_index()
# Choose the Part with the Lowest Min SLP to Keep
                if numpy.min(DF_After["SLP(hPa)"]) <= numpy.min(DF_Before["SLP(hPa)"]) and \
                len(DF_After) > len(DF_Before):
                    DF_Storm_Updated = DF_After
                else:
                    DF_Storm_Updated = DF_Before
                Track_Continuous = 1
                DF_Storm_Updated = DF_Storm_Updated.drop("level_0", axis=1)
# Repeat Until No More Track Discontinuity
    if Track_Continuous == 0:
        DF_Storm_Updated = DF_Storm.copy()
    return (DF_Storm_Updated, Track_Discon)

In [23]:
# Fix Storms That Have Time Discontinuity or Track Discontinuity
def DF_Fixing(DF_Storm_Orig, Track_Discons):
    Bottom_Right = False
    Top_Right = False
    Bottom_Left = False
    Top_Left = False
    Bottom_Right_k = []
    Top_Right_k = []
    Bottom_Left_k = []
    ET_Complete = False
#
# Check For Time Discontinuity
    DF_Storm = Time_Discontinuity(DF_Storm_Orig)
# Repeat to Check Again
    DF_Storm = Time_Discontinuity(DF_Storm)
    DF_Storm = Time_Discontinuity(DF_Storm)
#
# Check If Track Continuous
    DF_Storm, Track_Discon = Track_Discontinuity(DF_Storm)
    if len(Track_Discon) > 0:
        Track_Discons.append(Track_Discon)
# Repeat to Check Again
    DF_Storm, Track_Discon = Track_Discontinuity(DF_Storm)
    if len(Track_Discon) > 0:
        Track_Discons.append(Track_Discon)
    DF_Storm, Track_Discon = Track_Discontinuity(DF_Storm)
    if len(Track_Discon) > 0:
        Track_Discons.append(Track_Discon)
    return (DF_Storm, Track_Discons)

In [24]:
# Function to Find Whether a Storm Has Completed ET, and Start and End Time of ET
def ET_Function(DF_Storm):
    Tropical_Begin = False
    ET_Begin = False
    ET_Complete = False
    Bottom_Right_k = []
    Top_Right_k = []
    Bottom_Left_k = []
    Start_Time_k = -728
    End_Time_k = -728
    Trans_Type = -728
    for k in range(len(DF_Storm)):
        if ET_Complete == False:
            if Tropical_Begin == False:
# Storm Must First Be Tropical
                if DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
                    Tropical_Begin = True
                    Bottom_Right_k.append(k)
            if Tropical_Begin == True:
# Storm Must Be Tropical For At Least 48 Hours
                if len(Bottom_Right_k) <= 8:
                    if DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
                        Bottom_Right_k.append(k)
# Enter Next Stage If Storm Remains Tropical For At Least 48 Hours
                else:
                    if ET_Begin == False:
# If Storm Enters Top Right Quadrant = ET Begin
                        if DF_Storm["B"][k] > 15 and DF_Storm["VLT"][k] >= 0:
                            ET_Begin = True
                            Start_Time_k = k
                            Top_Right_k.append(k)
# If Storm Enters Bottom Left Quadrant = ET Begin
                        elif DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] < 0:
                            ET_Begin = True
                            Start_Time_k = k
                            Bottom_Left_k.append(k)
# If Storm Directly Enters Top Left Quadrant = Completed ET
                        elif DF_Storm["B"][k] > 15 and DF_Storm["VLT"][k] < 0:
                            ET_Complete = True
                            Start_Time_k = k
                            End_Time_k = k
                            Trans_Type = 3
                    elif ET_Begin == True:
                        if len(Top_Right_k) > 0:
# If Storm Remains in Top Right Quadrant
                            if DF_Storm["B"][k] > 15 and DF_Storm["VLT"][k] >= 0:
                                Top_Right_k.append(k)
# If Storm Enters Top Left or Bottom Left Quadrant = Completed ET
                            elif DF_Storm["VLT"][k] < 0:
                                ET_Complete = True
                                End_Time_k = k
                                Trans_Type = 1
                            elif DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
# If Storm Returns to Bottom Right Quadrant For Over 24 Hours
                                if k-4 > Top_Right_k[len(Top_Right_k)-1]:
                                    ET_Begin = False
                                    Top_Right_k = []
                        elif len(Bottom_Left_k) > 0:
# If Storm Remains in Bottom Left Quadrant
                            if DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] < 0:
                                Bottom_Left_k.append(k)
# If Storm Enters Top Left or Top Right Quadrant = Completed ET
                            elif DF_Storm["B"][k] > 15:
                                ET_Complete = True
                                End_Time_k = k
                                Trans_Type = 2
                            elif DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
# If Storm Returns to Bottom Right Quadrant For Over 24 Hours
                                if k-4 > Bottom_Left_k[len(Bottom_Left_k)-1]:
                                    ET_Begin = False
                                    Bottom_Left_k = []
#
# If Storm Did Not Spend At Least 48 Hours as Tropical
    if len(Bottom_Right_k) <= 8:
        Trans_Type = -728
# If ET Never Begun:
    elif End_Time_k == -728:
        if Start_Time_k == -728:
            Trans_Type = -1
# If ET Began But Did Not Complete:
        else:
            Trans_Type = 0
    return (Trans_Type, Start_Time_k, End_Time_k)

In [25]:
# Apply Function For Control DataFrames
Control_Data_V3, Control_ET_V1 = Storm_Info_DF(Control_Data_V2, Control_Codes_V2)

In [26]:
# Apply Function For RCP4.5 DataFrames
RCP45_Data_V3, RCP45_ET_V1 = Storm_Info_DF(RCP45_Data_V2, RCP45_Codes_V2)

In [27]:
# Apply Function For RCP8.5 DataFrames
RCP85_Data_V3, RCP85_ET_V1 = Storm_Info_DF(RCP85_Data_V2, RCP85_Codes_V2)

In [28]:
# Filter Out Storms That Have Not Been Tropical For At Least 48 Hours
def Tropical_Filter(Data_DF_Orig, ET_DF_Orig, Codes_DF_Orig):
    Filtered_ET = ET_DF_Orig[ET_DF_Orig["Trans Type"] > -728].reset_index()
    Filtered_ET = Filtered_ET.drop("index", axis=1)
    Code_List = Filtered_ET["Code"]
    Filtered_DF = Data_DF_Orig[Data_DF_Orig["Code"].isin(Code_List)].reset_index()
    Filtered_DF = Filtered_DF.drop("index", axis=1)
    Filtered_Codes = Codes_DF_Orig[Codes_DF_Orig["New Code"].isin(Code_List)].reset_index()
    Filtered_Codes = Filtered_Codes.drop("index", axis=1)
    return (Filtered_DF, Filtered_ET, Filtered_Codes)

In [29]:
# Apply Function
Control_Data_V4, Control_ET_V2, Control_Codes_V3 = Tropical_Filter(Control_Data_V3, Control_ET_V1, Control_Codes_V2)
RCP45_Data_V4, RCP45_ET_V2, RCP45_Codes_V3 = Tropical_Filter(RCP45_Data_V3, RCP45_ET_V1, RCP45_Codes_V2)
RCP85_Data_V4, RCP85_ET_V2, RCP85_Codes_V3 = Tropical_Filter(RCP85_Data_V3, RCP85_ET_V1, RCP85_Codes_V2)

In [30]:
# Add Storm Phase into DataFrame
def Storm_Phase_Info(Data_DF_Orig, ET_DF):
    Data_DF = Data_DF_Orig.copy()
    Storm_Phase_List = []
    for i in range(len(ET_DF)):
        DF_Storm = Data_DF[Data_DF["Code"] == ET_DF["Code"][i]].reset_index()
        Trans_Type = ET_DF["Trans Type"][i]
        Start_Time = ET_DF["ET Begin Time"][i]
        End_Time = ET_DF["ET Complete Time"][i]
# Find Storm Phase Based on ET Start Time and ET End Time
        Storm_Phase = Find_Storm_Phase(DF_Storm, Start_Time, End_Time, Trans_Type)
        for j in range(len(Storm_Phase)):
            Storm_Phase_List.append(Storm_Phase[j])
    Data_DF["Storm Phase"] = Storm_Phase_List
    Data_DF = Data_DF.drop("level_0", axis=1)
    return (Data_DF)

In [31]:
# Function to Find Storm Phase
def Find_Storm_Phase(DF_Storm, Start_Time, End_Time, Trans_Type):
    Time = list(DF_Storm["Time(Z)"])
    Storm_Phase = []
    for k in range(len(DF_Storm)):
# If Before ET Start = Tropical
# If Between ET Start and ET End = Transitioning
# If After ET End = Extratropical
        if Trans_Type == -1:
            Storm_Phase.append("Tropical")
        elif Trans_Type == 0:
            if Time[k] < Start_Time:
                Storm_Phase.append("Tropical")
            else:
                Storm_Phase.append("Transition")
        else:
            if Time[k] < Start_Time:
                Storm_Phase.append("Tropical")
            elif Time[k] < End_Time:
                Storm_Phase.append("Transition")
            else:
                Storm_Phase.append("Extratropical")
    return (Storm_Phase)

In [32]:
# Apply Function
Control_Data_V5 = Storm_Phase_Info(Control_Data_V4, Control_ET_V2)
RCP45_Data_V5 = Storm_Phase_Info(RCP45_Data_V4, RCP45_ET_V2)
RCP85_Data_V5 = Storm_Phase_Info(RCP85_Data_V4, RCP85_ET_V2)

In [33]:
Control_Data_V5[413:426]

,Code,Name,Lon,Lat,SLP(hPa),Winds(m/s),B,VLT,VUT,Time(Z),Storm Phase
413,TC190105,Lydia,-57.72,36.80,947.72,41.2,6.54,238.27,310.91,1901-09-26 18:00:00,Tropical
414,TC190105,Lydia,-56.33,37.40,948.48,41.5,9.05,236.10,289.92,1901-09-27 00:00:00,Tropical
415,TC190105,Lydia,-54.83,38.25,950.98,39.9,11.90,234.39,267.61,1901-09-27 06:00:00,Tropical
416,TC190105,Lydia,-53.37,39.19,953.76,43.1,18.13,224.89,230.05,1901-09-27 12:00:00,Transition
417,TC190105,Lydia,-51.56,40.08,952.91,42.0,31.98,231.25,195.68,1901-09-27 18:00:00,Transition
418,TC190105,Lydia,-49.70,42.10,951.29,38.4,43.81,253.98,172.97,1901-09-28 00:00:00,Transition
419,TC190105,Lydia,-49.51,44.55,948.89,44.0,41.29,241.12,161.34,1901-09-28 06:00:00,Transition
420,TC190105,Lydia,-49.52,44.98,949.44,43.2,38.45,220.08,156.22,1901-09-28 12:00:00,Transition
421,TC190105,Lydia,-48.37,45.66,952.82,37.7,28.80,201.31,169.85,1901-09-28 18:00:00,Transition
422,TC190105,Lydia,-47.67,46.28,959.65,33.9,14.50,153.80,168.18,1901-09-29 00:00:00,Transition


In [34]:
# Add ET Transition Lat Lon SLP Into ET DataFrame
def ET_Variables(Main_DF, ET_DF_Orig):
    ET_DF = ET_DF_Orig.copy()
    Array = numpy.zeros((15, len(ET_DF)))
    Peak_Times = []
    for i in range(len(ET_DF)):
        Code = ET_DF["Code"][i]
        Trans_Type = ET_DF["Trans Type"][i]
        Birth_Time = ET_DF["Genesis Time"][i]
        Start_Time = ET_DF["ET Begin Time"][i]
        End_Time = ET_DF["ET Complete Time"][i]
        Death_Time = ET_DF["Dissipate Time"][i]
        Storm = Main_DF[Main_DF["Code"] == Code]
#
# Find Lat, Lon, Time at Storm Peak
        Min_SLP = numpy.min(Storm["SLP(hPa)"])
        Storm_Peak = Storm[Storm["SLP(hPa)"] == Min_SLP]
        Array[0][i] = Min_SLP
        Array[1][i] = float(Storm_Peak["Lon"].iloc[0])
        Array[2][i] = float(Storm_Peak["Lat"].iloc[0])
        Peak_Time = Storm_Peak["Time(Z)"].iloc[0]
        Peak_Times.append(Peak_Time)
#
# Find SLP, Lon, Lat at Storm Genesis
        Birth = Storm[Storm["Time(Z)"] == Birth_Time]
        Array[3][i] = float(Birth["SLP(hPa)"].iloc[0])
        Array[4][i] = float(Birth["Lon"].iloc[0])
        Array[5][i] = float(Birth["Lat"].iloc[0])
#
# Find SLP, Lon, Lat at ET Begin
        if Trans_Type > -1:
            Begin = Storm[Storm["Time(Z)"] == Start_Time]
            Array[6][i] = float(Begin["SLP(hPa)"].iloc[0])
            Array[7][i] = float(Begin["Lon"].iloc[0])
            Array[8][i] = float(Begin["Lat"].iloc[0])
        else:
            Array[6][i] = numpy.nan
            Array[7][i] = numpy.nan
            Array[8][i] = numpy.nan
#
# Find SLP, Lon, Lat at ET Complete
        if Trans_Type > 0:
            Complete = Storm[Storm["Time(Z)"] == End_Time]
            Array[9][i] = float(Complete["SLP(hPa)"].iloc[0])
            Array[10][i] = float(Complete["Lon"].iloc[0])
            Array[11][i] = float(Complete["Lat"].iloc[0])
        else:
            Array[9][i] = numpy.nan
            Array[10][i] = numpy.nan
            Array[11][i] = numpy.nan
#
# Find SLP, Lon, Lat at Dissipation
        Death = Storm[Storm["Time(Z)"] == Death_Time]
        Array[12][i] = float(Death["SLP(hPa)"].iloc[0])
        Array[13][i] = float(Death["Lon"].iloc[0])
        Array[14][i] = float(Death["Lat"].iloc[0])
#
# Append Into ET DF
    ET_DF["Peak Time"] = Peak_Times
    ET_DF["Peak SLP"] = Array[0]
    ET_DF["Peak Lon"] = Array[1]
    ET_DF["Peak Lat"] = Array[2]
    ET_DF["Genesis SLP"] = Array[3]
    ET_DF["Genesis Lon"] = Array[4]
    ET_DF["Genesis Lat"] = Array[5]
    ET_DF["ET Begin SLP"] = Array[6]
    ET_DF["ET Begin Lon"] = Array[7]
    ET_DF["ET Begin Lat"] = Array[8]
    ET_DF["ET Complete SLP"] = Array[9]
    ET_DF["ET Complete Lon"] = Array[10]
    ET_DF["ET Complete Lat"] = Array[11]
    ET_DF["Dissipate SLP"] = Array[12]
    ET_DF["Dissipate Lon"] = Array[13]
    ET_DF["Dissipate Lat"] = Array[14]
    return (ET_DF)

In [35]:
# Apply Function
Control_ET_V5 = ET_Variables(Control_Data_V5, Control_ET_V2)
RCP45_ET_V5 = ET_Variables(RCP45_Data_V5, RCP45_ET_V2)
RCP85_ET_V5 = ET_Variables(RCP85_Data_V5, RCP85_ET_V2)

In [36]:
Control_ET_V5[5:10]

,Code,Name,Trans Type,Genesis Time,ET Begin Time,ET Complete Time,Dissipate Time,Peak Time,Peak SLP,Peak Lon,...,Genesis Lat,ET Begin SLP,ET Begin Lon,ET Begin Lat,ET Complete SLP,ET Complete Lon,ET Complete Lat,Dissipate SLP,Dissipate Lon,Dissipate Lat
5,TC190101,Gloria,0,1901-05-17 00:00:00,1901-05-24 12:00:00,NaT,1901-05-25 00:00:00,1901-05-22 18:00:00,962.99,-86.73,...,15.99,1000.16,-82.33,33.11,NaN,NaN,NaN,1010.66,-81.50,35.00
6,TC190102,Harrison,1,1901-05-23 00:00:00,1901-05-28 12:00:00,1901-05-30 18:00:00,1901-06-01 06:00:00,1901-05-27 06:00:00,980.89,-51.83,...,22.04,991.56,-43.71,30.04,1008.78,-34.75,24.00,1019.20,-38.00,21.50
7,TC190103,Janet,0,1901-07-08 18:00:00,1901-07-15 06:00:00,NaT,1901-07-16 12:00:00,1901-07-09 18:00:00,1001.93,-73.72,...,35.22,1013.44,-30.00,37.75,NaN,NaN,NaN,1019.59,-25.75,38.75
8,TC190104,Kinen,0,1901-09-08 18:00:00,1901-09-11 00:00:00,NaT,1901-09-15 18:00:00,1901-09-13 06:00:00,998.95,-56.51,...,28.57,1002.26,-63.91,33.16,NaN,NaN,NaN,1002.47,-47.75,50.50
9,TC190105,Lydia,1,1901-09-11 12:00:00,1901-09-27 12:00:00,1901-09-29 18:00:00,1901-09-29 18:00:00,1901-09-24 00:00:00,913.33,-60.97,...,13.13,953.76,-53.37,39.19,970.00,-42.82,48.53,970.00,-42.82,48.53


In [37]:
# Filter For Just Storms That Completed ET Transition
def ET_Filter(Data_DF_Orig, ET_DF_Orig):
    ET_DF = ET_DF_Orig[ET_DF_Orig["Trans Type"] > 0].reset_index()
    ET_DF = ET_DF.drop("index", axis=1)
    Code_List = ET_DF["Code"]
    Filtered_DF = Data_DF_Orig[Data_DF_Orig["Code"].isin(Code_List)].reset_index()
    Filtered_DF = Filtered_DF.drop("index", axis=1)
    return (Filtered_DF, ET_DF)

In [38]:
# Apply Function
Control_Data_V6, Control_ET_V6 = ET_Filter(Control_Data_V5, Control_ET_V5)
RCP45_Data_V6, RCP45_ET_V6 = ET_Filter(RCP45_Data_V5, RCP45_ET_V5)
RCP85_Data_V6, RCP85_ET_V6 = ET_Filter(RCP85_Data_V5, RCP85_ET_V5)

In [39]:
Control_Data_V6[280:293]

,Code,Name,Lon,Lat,SLP(hPa),Winds(m/s),B,VLT,VUT,Time(Z),Storm Phase
280,TC190105,Lydia,-57.72,36.80,947.72,41.2,6.54,238.27,310.91,1901-09-26 18:00:00,Tropical
281,TC190105,Lydia,-56.33,37.40,948.48,41.5,9.05,236.10,289.92,1901-09-27 00:00:00,Tropical
282,TC190105,Lydia,-54.83,38.25,950.98,39.9,11.90,234.39,267.61,1901-09-27 06:00:00,Tropical
283,TC190105,Lydia,-53.37,39.19,953.76,43.1,18.13,224.89,230.05,1901-09-27 12:00:00,Transition
284,TC190105,Lydia,-51.56,40.08,952.91,42.0,31.98,231.25,195.68,1901-09-27 18:00:00,Transition
285,TC190105,Lydia,-49.70,42.10,951.29,38.4,43.81,253.98,172.97,1901-09-28 00:00:00,Transition
286,TC190105,Lydia,-49.51,44.55,948.89,44.0,41.29,241.12,161.34,1901-09-28 06:00:00,Transition
287,TC190105,Lydia,-49.52,44.98,949.44,43.2,38.45,220.08,156.22,1901-09-28 12:00:00,Transition
288,TC190105,Lydia,-48.37,45.66,952.82,37.7,28.80,201.31,169.85,1901-09-28 18:00:00,Transition
289,TC190105,Lydia,-47.67,46.28,959.65,33.9,14.50,153.80,168.18,1901-09-29 00:00:00,Transition


In [40]:
Control_ET_V6[5:10]

,Code,Name,Trans Type,Genesis Time,ET Begin Time,ET Complete Time,Dissipate Time,Peak Time,Peak SLP,Peak Lon,...,Genesis Lat,ET Begin SLP,ET Begin Lon,ET Begin Lat,ET Complete SLP,ET Complete Lon,ET Complete Lat,Dissipate SLP,Dissipate Lon,Dissipate Lat
5,TC190105,Lydia,1,1901-09-11 12:00:00,1901-09-27 12:00:00,1901-09-29 18:00:00,1901-09-29 18:00:00,1901-09-24 00:00:00,913.33,-60.97,...,13.13,953.76,-53.37,39.19,970.00,-42.82,48.53,970.00,-42.82,48.53
6,TC190106,Mario,1,1901-09-19 00:00:00,1901-09-22 00:00:00,1901-09-22 18:00:00,1901-09-23 18:00:00,1901-09-22 12:00:00,988.80,-74.39,...,26.29,993.69,-78.47,35.78,991.98,-71.20,41.68,1007.97,-57.75,48.00
7,TC190107,Nicole,2,1901-10-04 06:00:00,1901-10-14 12:00:00,1901-10-15 00:00:00,1901-10-15 06:00:00,1901-10-05 18:00:00,974.56,-72.08,...,31.01,1015.94,-22.00,42.00,1019.52,-26.00,41.25,1019.68,-27.50,41.25
8,TC190109,Shannon,1,1901-12-10 06:00:00,1901-12-14 00:00:00,1901-12-14 06:00:00,1901-12-14 18:00:00,1901-12-11 06:00:00,942.03,-48.79,...,43.50,975.20,-32.75,47.00,975.97,-30.25,47.75,975.85,-25.00,52.00
9,TC190201,Tony,2,1902-01-25 06:00:00,1902-01-27 18:00:00,1902-01-28 00:00:00,1902-01-28 12:00:00,1902-01-25 06:00:00,1002.71,-32.02,...,28.50,1012.65,-43.50,29.50,1014.76,-44.50,28.00,1016.34,-47.00,25.00


In [41]:
# Filter For Just Storms With Peak SLP < 1000
def SLP_Filter(Data_DF_Orig, ET_DF_Orig):
    ET_DF = ET_DF_Orig[ET_DF_Orig["Peak SLP"] < 1000].reset_index()
    ET_DF = ET_DF.drop("index", axis=1)
    Code_List = ET_DF["Code"]
    Filtered_DF = Data_DF_Orig[Data_DF_Orig["Code"].isin(Code_List)].reset_index()
    Filtered_DF = Filtered_DF.drop("index", axis=1)
    return (Filtered_DF, ET_DF)

In [42]:
# Apply Function
Control_Data_V7, Control_ET_V7 = SLP_Filter(Control_Data_V6, Control_ET_V6)
RCP45_Data_V7, RCP45_ET_V7 = SLP_Filter(RCP45_Data_V6, RCP45_ET_V6)
RCP85_Data_V7, RCP85_ET_V7 = SLP_Filter(RCP85_Data_V6, RCP85_ET_V6)

In [43]:
# Output DF to csv File
def Output_File(DF, Model, Type):
    File_Name = str(Model+'_'+Type+'_Output_V6.csv')
    DF.to_csv(Output_Diri+File_Name)

In [44]:
# Output Control Files
Output_File(Control_Data_V5, "Control", "Data_All")
Output_File(Control_Data_V6, "Control", "Data_ExTC")
Output_File(Control_Data_V7, "Control", "Data_hPa")
Output_File(Control_ET_V5, "Control", "ET_All")
Output_File(Control_ET_V6, "Control", "ET_ExTC")
Output_File(Control_ET_V7, "Control", "ET_hPa")
Output_File(Control_Codes_V3, "Control", "Codes")

In [45]:
# Output RCP4.5 Files
Output_File(RCP45_Data_V5, "RCP45", "Data_All")
Output_File(RCP45_Data_V6, "RCP45", "Data_ExTC")
Output_File(RCP45_Data_V7, "RCP45", "Data_hPa")
Output_File(RCP45_ET_V5, "RCP45", "ET_All")
Output_File(RCP45_ET_V6, "RCP45", "ET_ExTC")
Output_File(RCP45_ET_V7, "RCP45", "ET_hPa")
Output_File(RCP45_Codes_V3, "RCP45", "Codes")

In [46]:
# Output RCP8.5 Files
Output_File(RCP85_Data_V5, "RCP85", "Data_All")
Output_File(RCP85_Data_V6, "RCP85", "Data_ExTC")
Output_File(RCP85_Data_V7, "RCP85", "Data_hPa")
Output_File(RCP85_ET_V5, "RCP85", "ET_All")
Output_File(RCP85_ET_V6, "RCP85", "ET_ExTC")
Output_File(RCP85_ET_V7, "RCP85", "ET_hPa")
Output_File(RCP85_Codes_V3, "RCP85", "Codes")